### Experiment #1 (Model comparison)
- Script: mamdani_tsk_selection.py
- Raw data generation command:
```bash
	$ bin/mamdani_tsk_selection
```
- Raw data result: Mamdani_TSK_Selection.xlsx
- Time required: total +/- 30 minutes (12 sheets x 4 model x 3 clustering algorithm X 3 MFs)

#### Pivot Table (Summary)

In [1]:
from notebook_resolver import *
import pandas as pd

file_path = "experiments/Mamdani_TSK_Selection.xlsx"

df = pd.read_excel(file_path, sheet_name="fis_selection")
# Convert R2 column to float (replace comma with dot first)
df["R2"] = df["R2"].astype(str).str.replace(",", ".").astype(float)

# Clean Clustering Method names for consistency
df["Clustering Method"] = df["Clustering Method"].str.replace("ClusteringMethod.", "")

# Create pivot table like in the picture
pivot = pd.pivot_table(
    df,
    values="R2",
    index=["FIS Type", "MFs Type"],
    columns="Clustering Method",
    aggfunc="mean"
)

# Rename columns to match picture style
pivot = pivot.rename(columns={
    "FUZZY_C_MEANS": "Average of R² (FCM)",
    "KMEANS": "Average of R² (K-Means)",
    "MBKMEANS": "Average of R² (Mini-batch K-Means)"
})

pivot = pivot.reset_index().round(4)

pivot

Clustering Method,FIS Type,MFs Type,Average of R² (FCM),Average of R² (K-Means),Average of R² (Mini-batch K-Means)
0,it2_mamdani,MFType2.GAUSSIAN,0.6323,0.6317,0.6668
1,it2_mamdani,MFType2.TRAPEZOIDAL,-0.0674,-0.0351,0.0988
2,it2_mamdani,MFType2.TRIANGULAR,0.1705,0.2281,0.3773
3,it2_tsk,MFType2.GAUSSIAN,0.9761,0.9761,0.9799
4,it2_tsk,MFType2.TRAPEZOIDAL,0.5007,0.0775,0.8175
5,it2_tsk,MFType2.TRIANGULAR,-0.0092,-1.3780,-0.7889
6,mamdani,MFType1.GAUSSIAN,-0.8060,-0.7184,-0.7879
7,mamdani,MFType1.TRAPEZOIDAL,0.3670,0.1694,0.2550
8,mamdani,MFType1.TRIANGULAR,0.3730,0.2333,0.2550
9,tsk,MFType1.GAUSSIAN,0.9439,0.8849,0.8687


In [2]:
from src.utils.excel_operation import export_to_excel

export_to_excel(pivot, file_path, sheet_name="pivot_table")


### Selected Model (IT2TSK - MBK-Means - Gaussian MF)

In [3]:
df = pd.read_excel(file_path, sheet_name="fis_selection")

selected_df = df[
    (df["FIS Type"] == "it2_tsk") & 
    (df["Clustering Method"] == "ClusteringMethod.MBKMEANS") &
    (df["MFs Type"] == "MFType2.GAUSSIAN")]


selected_model_df = selected_df[[
    "Worksheet Name",
    "R2",
    "MSE",
    "MAE",
    "total_rule_base_clustering_time",
    "total_rules",
    "total_features_clustering_time",
    "total_features"]].round(4)

# Calculate averages and totals
avg_r2 = selected_model_df["R2"].mean()
avg_mse = selected_model_df["MSE"].mean()
avg_mae = selected_model_df["MAE"].mean()
avg_rule_base_time = selected_model_df["total_rule_base_clustering_time"].mean()
avg_features_time = selected_model_df["total_features_clustering_time"].mean()

# Create a summary row as a DataFrame
summary_row = pd.DataFrame({
    "Worksheet Name": ["Average"],
    "R2": [avg_r2],
    "MSE": [avg_mse],
    "MAE": [avg_mae],
    "total_rule_base_clustering_time": [avg_rule_base_time],
    "total_rules": [""],
    "total_features_clustering_time": [avg_features_time],
    "total_features": [""]
})

# Append to the original DataFrame
final_df = pd.concat([selected_model_df, summary_row], ignore_index=True)

# Round numeric columns for presentation
final_df[["R2", "MSE", "MAE", 
          "total_rule_base_clustering_time", 
          "total_features_clustering_time"]] = final_df[[
              "R2", "MSE", "MAE", 
              "total_rule_base_clustering_time", 
              "total_features_clustering_time"
          ]].astype(float).round(4)

# Display result
final_df

export_to_excel(final_df, file_path, "selected_it2tsk")

final_df

,Worksheet Name,R2,MSE,MAE,total_rule_base_clustering_time,total_rules,total_features_clustering_time,total_features
0,1.Inside-Outside,0.9776,0.0020,0.0328,0.6982,2,5.2767,12
1,2.Round,0.9917,0.0007,0.0189,0.7885,2,5.5890,12
2,3.Top_Sirloin,0.9745,0.0022,0.0304,0.7824,2,5.3380,12
3,4.Tenderloin,0.9789,0.0018,0.0242,0.6852,3,5.3738,12
4,5.Flap_meat,0.9858,0.0012,0.0254,0.6321,2,7.0702,12
5,6.Striploin,0.9894,0.0009,0.0217,0.7923,2,5.5180,12
6,7.Rib_eye,0.9923,0.0007,0.0188,4.7033,2,6.0533,12
7,8.Skirt_meat,0.9945,0.0005,0.0140,0.6926,3,5.2510,12
8,9.Brisket,0.9919,0.0007,0.0175,0.8042,2,5.2768,12
9,10.Clod_Chuck,0.9450,0.0047,0.0419,0.6921,2,5.6115,12
